In [1]:
from Survival.FeatureEngineer import FeatureEngineer
from Survival.Utils import model_prepare
from Survival.Utils import evaluate_predict_result
from Survival.IPEC import IPEC
from Survival.WeibullRegressionModel import WeibullRegressionModel


Using TensorFlow backend.


In [2]:
fe = FeatureEngineer(verbose=False, data_path="../../dataset/")
sources = fe.get_diseases_list()

dataset_idx = 0
file_prefix = ""
low_event_thd = 0.03
low_value_thd = low_event_thd / 3
patient_dict, feature_set, train_id_list, test_id_list = \
    fe.load_data_as_dict(dataset_idx, file_prefix=file_prefix, 
                         low_freq_event_thd=low_event_thd, 
                         low_freq_value_thd=low_value_thd)
train_df, test_df, feature_list = \
    model_prepare(patient_dict, feature_set, train_id_list, test_id_list)


/usr/local/lib/python3.6/site-packages/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars
  return (np.sqrt(ssd) / old_norm) < self.convergence_threshold


In [3]:
model = WeibullRegressionModel(pca_flag=True, n_components=17)


In [4]:
model.fit(train_df)


In [5]:
test_time_median_pred = model.pred_median_time(test_df)
concordance = evaluate_predict_result(test_time_median_pred, test_df, print_result=True)


concordance: 0.6084567606306737


In [6]:
ipec = IPEC(train_df, g_type="All_One", t_thd=0.8, t_step="obs", time_col='LOS', death_identifier='OUT', verbose=False)
check_points = ipec.get_check_points()


In [7]:
proba_matrix = model.pred_proba(test_df, time=check_points)


In [9]:
ipec_score = ipec.calc_ipec(proba_matrix, list(test_df["LOS"]), list(test_df["OUT"]))
ipec_score


1.7118722378790512